# ショットガンの検討
これまでにわかったこと
- 出来高（tick数）が多いほどテクニカル分析が効きやすそう
- 1銘柄をずっと持ち続けた場合とreturnは大きく変わらない?
- 最終的には目で見ての判断がまだ必要か

これから検証すること
- パラメータのチューニング（短期・中期・長期、最大保有期間)
- 売買システムの構築

In [ ]:
from datetime import datetime
from pathlib import Path
from typing import List
import csv

import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

import stock

pio.renderers.default = "notebook"

In [ ]:
DAILY_DATA_DIR = Path.home() / "remote/gdrive/stock/data/daily/"
TARGET_LIST_CSV = stock.DATA_DIR / "margin_trade_target_list.csv"
daily_data_files = sorted(DAILY_DATA_DIR.glob("20220218_*.csv"))

with open(TARGET_LIST_CSV) as f:
    reader = csv.reader(f)
    next(reader)
    target_list = [row[0] for row in reader]

In [ ]:
def load_10year_data(code: str) -> pd.DataFrame:
    csv_path = stock.DATA_DIR / "etfs"/ f"{code}.csv"
    if not csv_path.exists():
        csv_path = stock.DATA_DIR / "nikkei225"/ f"{code}.csv"
    df = pd.read_csv(csv_path)
    df["day"] = df["timestamp"].map(lambda x: datetime.fromtimestamp(x).strftime("%Y/%m/%d"))
    df["open"] = df["start"]
    df["close"] = df["end"]
    return df

In [ ]:
shotgun_params = stock.autotrade.shotgun.ShotgunParams(
    short_term = 5,
    mid_term= 25,
    long_term = 75,
    max_hold_days = 9
)
shotgun = stock.autotrade.shotgun.Shotgun(shotgun_params)

In [ ]:
code = target_list[10]
df = pd.read_csv(DAILY_DATA_DIR / f"20220218_{code}.csv")
results = shotgun.run(df, with_plot=True)
profit = [r.profit for r in results]
print(f"profit: {sum(profit)}")
for r in results:
    print("buy = {}, sell = {}, profit = {:.2f},  buy = {:.1f},  sell = {:.1f}".format(
        df["day"][r.buy_index], df["day"][r.sell_index], r.profit, r.buy, r.sell
    ))

In [ ]:
idx = 1
code = target_list[idx]
#df = load_10year_data(code) #pd.read_csv(DAILY_DATA_DIR / f"20220218_{code}.csv")
df = pd.read_csv(DAILY_DATA_DIR / f"20220218_{code}.csv")

short_term = [5]
mid_term = [15, 20, 25]
long_term = [60, 80, 100]
best = 0
total = 0

for st in short_term:
    for mt in mid_term:
        for lt in long_term:
            shotgun_params = stock.autotrade.shotgun.ShotgunParams(
                short_term = st,
                mid_term= mt,
                long_term = lt,
            )
            shotgun = stock.autotrade.shotgun.Shotgun(shotgun_params)
            results = shotgun.run(df, with_plot=False)
            profit = sum([r.profit for r in results])
            total += profit
            if profit > best:
                best = profit
                print(f"profit: {profit}, best: {st}, {mt}, {lt}")

print("total = {} ".format(total))

In [ ]:
df = pd.read_csv(DAILY_DATA_DIR / f"20220218_{code}.csv")

short_term = [3, 5, 7]
mid_term = [15, 20, 25]
long_term = [60, 75, 90]
best = 0

for st in short_term:
    for mt in mid_term:
        for lt in long_term:
            shotgun_params = stock.autotrade.shotgun.ShotgunParams(
                short_term = st,
                mid_term= mt,
                long_term = lt,
            )
            shotgun = stock.autotrade.shotgun.Shotgun(shotgun_params)
            results = shotgun.run(df, with_plot=False)
            profit = sum([r.profit for r in results])
            if profit > best:
                best = profit
                print(f"profit: {profit}, best: {st}, {mt}, {lt}")

In [ ]:
## 条件を満たした場合に売り -> 翌日朝に買い直しのほうが良いのでは？

In [ ]:
class InverseShotgun(stock.autotrade.shotgun.Shotgun):
    """買いシグナルが出たら引けで売り、次の日の始値で買い直し
    """
    def _calc_sell_indices(self, df: pd.DataFrame, buy_indices: List[int]) -> List[int]:
        return buy_indices

In [ ]:
shotgun_params = stock.autotrade.shotgun.ShotgunParams(
    short_term = 5,
    mid_term= 20,
    long_term = 60,
    max_hold_days = 9
)
inv_shotgun = InverseShotgun(shotgun_params)

In [ ]:
code = target_list[1]
df = pd.read_csv(DAILY_DATA_DIR / f"20220218_{code}.csv")
results = inv_shotgun.run(df, with_plot=True)
profit = [r.profit for r in results]
print(f"profit: {sum(profit)}")
for r in results:
    print("buy = {}, sell = {}, profit = {:.2f},  buy = {:.1f},  sell = {:.1f}".format(
        df["day"][r.buy_index], df["day"][r.sell_index], r.profit, r.buy, r.sell
    ))